In [1]:
import numpy as np
%matplotlib qt 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import pickle
import utils

In [2]:
topdown_points = np.loadtxt('topdown1_undistorted_image_points.txt')
topdown_points = np.array(topdown_points, dtype=np.float32)
fronton_points = np.loadtxt('fronton1_undistorted_image_points.txt')
fronton_points = np.array(fronton_points, dtype=np.float32)
world_points = np.loadtxt('world_points_3D.txt')
world_points = np.array(world_points, dtype=np.float32)
topdown_camera = pickle.load(open('topdown1_camera_object.pkl', 'rb'))
fronton_camera = pickle.load(open('fronton1_camera_object.pkl', 'rb'))


In [3]:
# Set Rotation and Translation Matrices

topdown_camera.set_rotation_and_translation_vectors(world_points, topdown_points)
fronton_camera.set_rotation_and_translation_vectors(world_points, fronton_points)

In [4]:
# Triangulate points 

triangulated_points = utils.triangulate(topdown_points, topdown_camera, fronton_points, fronton_camera)

# Plot 3D points

utils.visualize_triangulated_points(triangulated_points, world_points)

# Get triangulation error

print('Triangulation error: ', np.linalg.norm(triangulated_points - world_points, axis=0).mean())
    

Triangulation error:  0.12082623723614272


In [13]:
# Select new points to triangulate

imgpath = 'fronton1.png'
image = cv2.imread(imgpath)
# Interactive window to collect points from the image
fronton_selected = []
fronton_selected = utils.interactive_clicking_and_saving(image, fronton_selected)


In [14]:

utils.display_image_with_points(image, fronton_selected)

In [15]:
# Select new points to triangulate

imgpath = 'topdown1.png'
image = cv2.imread(imgpath)
# Interactive window to collect points from the image
topdown_selected = []
topdown_selected = utils.interactive_clicking_and_saving(image, topdown_selected)

In [16]:
utils.display_image_with_points(image, topdown_selected)

In [17]:
# Dewarp both points

dewarped_topdown_selected = cv2.undistortPoints(np.array([topdown_selected]), topdown_camera.camera_matrix, topdown_camera.distortion_coefficients, P=topdown_camera.new_camera_matrix).reshape(-1,2)
dewarped_fronton_selected = cv2.undistortPoints(np.array([fronton_selected]), fronton_camera.camera_matrix, fronton_camera.distortion_coefficients, P=fronton_camera.new_camera_matrix).reshape(-1,2)

triangulated_points = utils.triangulate(dewarped_topdown_selected, topdown_camera, dewarped_fronton_selected, fronton_camera)

# Visualize the new triangulated points
NET_Y = 0.76 + 3.96 + 1.98
NET_HEIGHT = 1.55
RIGHT_BOUNDARY_X = 0.46 + 2.59 + 2.59 + 0.46
new_world_points = np.array([[0 ,NET_Y, NET_HEIGHT],[ RIGHT_BOUNDARY_X,NET_Y ,NET_HEIGHT]])

utils.visualize_triangulated_points(triangulated_points, new_world_points)

# Get triangulation error

print('Triangulation error: ', np.linalg.norm(triangulated_points - new_world_points, axis=0).mean())


Triangulation error:  0.20104567310344512
